### Nested decoding of subjects from the chocolate study
### Decoding of response to app vs. unapp images using an ANOVA-based feature selection, within the power atlas


In [3]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
from nilearn.image import concat_imgs, index_img, smooth_img
from nilearn.image import resample_to_img
#from nilearn import plotting
from nilearn.input_data import NiftiMasker
from sklearn.svm import SVC
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score, permutation_test_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from nilearn import image
#from nilearn.plotting import plot_stat_map, show
from sklearn.dummy import DummyClassifier

# ---STEP 1---
### Concatenate the imagine data into a NIFTI-2 file. 
### Note: the data does not fit into a NIFTI-1 file, due to large number of subs. 


In [4]:
#set basepath
basepath= '/Users/nikkibytes/Documents/testing/Nilearn_projects/ChocoData'

#make a list of the files to concat
all_func = glob.glob(os.path.join(basepath, 'cs*++.feat', 'filtered_func_data.nii.gz'))

#load in all the files from the glob above, then convert them from nifti1 to nifti2
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in all_func)
#concat, this is with nibabel, but should work with nilearn too
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
#set the output file name
outfile=os.path.join(basepath,'concatenated_imagine.nii')
#write the file
ni2_concat.to_filename(outfile)


sizeof_hdr should be 540; set sizeof_hdr to 540
sizeof_hdr should be 540; set sizeof_hdr to 540


# ---STEP 2---
#### load & prepare MRI data

In [26]:

#load, fxnl, anatomical, & mask for plotting
fmri_subjs=os.path.join(basepath, 'concatenated_imagine.nii')
average_ana=os.path.join(basepath,'CS_avg_mprage_image.nii.gz')
imag_mask=os.path.join(basepath,'power_roimask_4bi.nii.gz')

#plot mask (Power ROIs) over anatomical that is defined above
#plotting.plot_roi(imag_mask,bg_img=average_ana,cmap='Paired')

#load labels for the functional data
stim = os.path.join(basepath,'label_67_sub.csv')
labels = np.recfromcsv(stim, delimiter=",", encoding='UTF-8')
#print(labels)
#Its shape corresponds to the number of time-points times the number of voxels in the mask

#Retrieve the behavioral targets, that we are going to predict in the decoding
y_mask = labels['labels']
subs = labels['subs']
print("y mask shape:", y_mask.shape)

y mask shape: (31825,)


# ---STEP 3---
#### feature selection

In [27]:

#To keep only data corresponding to app food or unapp food, we create a mask of the samples belonging to the condition.
condition_mask = np.logical_or(y_mask == 'app',y_mask == 'unapp')
print("CONDITION MASK SIZE:", condition_mask.shape)
y = y_mask[condition_mask]
print("Y shape:", y.shape)
n_conditions = np.size(np.unique(y))

#prepare the fxnl data. 
nifti_masker = NiftiMasker(mask_img=imag_mask,
                           smoothing_fwhm=4,standardize=True,
                           memory="nilearn_cache",memory_level=1)

fmri_trans = nifti_masker.fit_transform(fmri_subjs)
#print("FMRI TRANS: ",fmri_trans[0])
print("FMRI TRANS SHAPE: ", fmri_trans.shape)
#X = fmri_trans[condition_mask]
#subs = subs[condition_mask]

CONDITION MASK SIZE: (31825,)
Y shape: (7973,)
FMRI TRANS SHAPE:  (950, 6223)
